In [7]:
import glob
import numpy as np 
import mne
import pandas as pd
import os
import pickle

In [ ]:
dataset_path = '/data/zanola/BrainLats/'

file_paths = glob.glob(dataset_path+"*/*/sub-*/eeg/*.set")
dataset_lookup = {}
for path in file_paths:
    parts = os.path.normpath(path).split(os.sep)
    coorte = parts[-5]
    soggetto = parts[-3]
    dataset_lookup[(coorte, soggetto)] = path

In [ ]:
#Salvo tutti i file.set di CN,FTD,AD
file_list_CN= glob.glob(dataset_path+'5_HC/*/sub-*/eeg/*.set')
file_list_FTD= glob.glob(dataset_path+'2_bvFTD/*/sub-*/eeg/*.set')
file_list_AD = glob.glob(dataset_path+'1_AD/*/sub-*/eeg/*.set')

#Funzione che analizza file
def analizza_file_eeglab(f):
    raw = mne.io.read_raw_eeglab(f, preload=False)
    fdt_paths = [fname for fname in raw.filenames if fname and fname.endswith('.fdt')]
    return {
        'is_standalone': len(fdt_paths) == 0,
        'fdt_path': fdt_paths[0] if fdt_paths else 'NaN'
    }
#Funzione che carica file:
list_CN=[]
list_FTD=[]
list_AD=[]
info_CN=[]
info_FTD=[]
info_AD=[]
def carica_file(file_list, lista_dati, lista_info, gruppo):
    log_errori = []

    for f in file_list:
        try:
            if analizza_file_eeglab(f)['is_standalone']:
                data = mne.io.read_raw_eeglab(f, preload=True,verbose=False)
                status = 'standalone'
                referenced_fdt = analizza_file_eeglab(f)['fdt_path']
            else:
                data = mne.io.read_raw_eeglab(f, preload=False,verbose=False)
                data.load_data()
                status = 'non-standalone'
                referenced_fdt = analizza_file_eeglab(f)['fdt_path']

        except Exception as e:
            log_errori.append({
                'file': f,
                'errore': str(e),
                'status': 'non-standalone ma linka a file.fdt inesistente',
                'path_fdt': referenced_fdt,
                'group': gruppo
            })
            #print(f"Errore con {f}: {e}")
            continue

        lista_dati.append(data)
        lista_info.append({
            'file': f,
            'status': status,
            'path_fdt': referenced_fdt,
            'group': gruppo
        })
    return log_errori   
log_errori_CN = carica_file(file_list_CN, list_CN, info_CN, 'CN')
log_errori_FTD = carica_file(file_list_FTD, list_FTD, info_FTD, 'FTD')
log_errori_AD = carica_file(file_list_AD, list_AD, info_AD, 'AD')

#Creo dataframe con file con errori, dataframe validi, dataframe completo(ordinato in base all'ordine dei file originali)
df_errori = pd.DataFrame(log_errori_CN + log_errori_FTD + log_errori_AD)

df_validi = pd.DataFrame(info_CN + info_FTD + info_AD)

df_completo = pd.concat([df_validi, df_errori], ignore_index=True)
file_list_tutti = file_list_CN + file_list_FTD + file_list_AD
df_completo['ordine'] = df_completo['file'].apply(lambda x: file_list_tutti.index(x) if x in file_list_tutti else -1)
df_completo = df_completo.sort_values('ordine').reset_index(drop=True)
df_completo = df_completo.drop(columns='ordine')

#Visualizzo dataframe completo
display(df_completo)

In [ ]:
path.split(os.sep)[4][0]

'5_HC'

In [ ]:
pickle_path = '/data/zanola/BrainLats_pickle/'

a_id = []
c_id = []
f_id = []

for i in range(len(df_validi)):
    raw=mne.io.read_raw_eeglab(df_validi['file'][i], verbose = False,
                            preload=True)

    raw = raw.resample(125)

    print(np.shape(raw.get_data())[0])

    EEGpickle = {}
    EEGpickle['data'] = raw.get_data()*(10**6)
    EEGpickle['label' ]= conv[df_validi['group'][i]]

    index = file_paths.index(df_validi['file'][i])

    if df_validi['group'][i]=='CN':
        c_id.append(index)
    elif df_validi['group'][i]=='AD':
        a_id.append(index)
    else:
        f_id.append(index)

    string = '26_' + str(index+1) + '_1_1.pkl'

    # with open(pickle_path+string, "wb") as f:
    #     pickle.dump(EEGpickle, f)



In [10]:
pickle_path = '/data/zanola/BrainLats_pickle/icasr/'
string = '26_301_1_1.pickle'
with open(pickle_path+string, "rb") as f:
    data = pickle.load(f)

data['label']

2

In [ ]:
df_validi

In [53]:
int(label[0])

1

In [77]:
root_path = '/data/zanola/'
save_path = '/data/zanola/BrainLats_pickle/icasr/'
ds_code = 'brainlat'
conv = {'5_HC':0,'2_bvFTD':1,'1_AD':2, '3_PD': 3, '4_MS': 4}

eegpath = glob.glob(root_path + 'BrainLats/**/**/**/**/*.set')
eegpath = sorted(
    eegpath, 
    key = lambda x: (int(x.split(os.sep)[4][0]), int(x.split(os.sep)[6][-2:]))
)

path_problem = []
print("start reading EEG from brainlat")
for path in eegpath:
    try:
        # Load EEG from .set
        data = mne.io.read_raw_eeglab(path, preload=True, verbose=False)

        # resample to 250 HZ
        data = data.resample(250)

        # extract array and select only 61 channels from 10/10 template
        data = data.get_data()
        # data = data[channel_extraction_order]

        # remove first 8 and last 8 seconds
        # data = data[:,250*8:-250*8]

        # label is an integer from 0 to 4
        label = path.split(os.sep)[4]

        # subject id is a 3-digit integer
        # label+1 - last 2 digits of the original subject ID
        subject_id = str(conv[label]+1) + path.split(os.sep)[6][-2:]

        # add dataset ID = 26 and pickle extension
        file_name = "_".join( ("26", subject_id, "1", "1") )


        # double-check print
        print(path, file_name, label, subject_id, data.shape)
        
        # store pickle as usual
        eeg   = {'data': data, 'label': conv[label] }
        with open(save_path + file_name + '.pickle', 'wb') as handle:
            pickle.dump(eeg, handle, protocol = pickle.HIGHEST_PROTOCOL)
    
    except Exception:
        # Some EEGs are broken. We keep track of them
        # ask brainlat curators why they included rejected or broken EEG
        # I don't know
        print(f"Cannot read file at path {path[22:]}")
        path_problem.append(path)

start reading EEG from brainlat
/data/zanola/BrainLats/1_AD/AR/sub-30001/eeg/s6_sub-30001_rs-HEP_eeg.set 26_301_1_1 1_AD 301 (128, 78111)
/data/zanola/BrainLats/1_AD/AR/sub-30002/eeg/s6_sub-30002_rs-HEP_eeg.set 26_302_1_1 1_AD 302 (128, 80827)
/data/zanola/BrainLats/1_AD/CL/sub-30003/eeg/s6_sub-30003_rs-HEP_eeg.set 26_303_1_1 1_AD 303 (128, 109598)
/data/zanola/BrainLats/1_AD/AR/sub-30004/eeg/s6_sub-30004_rs-HEP_eeg.set 26_304_1_1 1_AD 304 (128, 71251)
/data/zanola/BrainLats/1_AD/CL/sub-30005/eeg/s6_sub-30005_rs-HEP_eeg.set 26_305_1_1 1_AD 305 (128, 145378)
/data/zanola/BrainLats/1_AD/CL/sub-30006/eeg/s6_sub-30006_rs-HEP_eeg.set 26_306_1_1 1_AD 306 (128, 155066)
/data/zanola/BrainLats/1_AD/CL/sub-30007/eeg/s6_sub-30007_rs-HEP_eeg.set 26_307_1_1 1_AD 307 (128, 126362)
/data/zanola/BrainLats/1_AD/AR/sub-30008/eeg/s6_sub-30008_rs-HEP_eeg.set 26_308_1_1 1_AD 308 (128, 78580)
/data/zanola/BrainLats/1_AD/AR/sub-30009/eeg/s6_sub-30009_rs-HEP_eeg.set 26_309_1_1 1_AD 309 (128, 101090)
/data/zan

/tmp/ipykernel_2331099/1742952222.py:17: RuntimeWarning: Data file name in EEG.data (s6_sub-10005_rs_eeg.fdt) is incorrect, the file name must have changed on disk, using the correct file name (sub-40005_rs_eeg.fdt).
  data = mne.io.read_raw_eeglab(path, preload=True, verbose=False)


/data/zanola/BrainLats/3_PD/CL/sub-40005/eeg/sub-40005_rs_eeg.set 26_405_1_1 3_PD 405 (128, 128500)
Cannot read file at path /3_PD/CL/sub-40006/eeg/s40006_CH_PD_reject.set
Cannot read file at path /3_PD/AR/sub-40006/eeg/s40006_AR_PD_reject.set


/tmp/ipykernel_2331099/1742952222.py:17: RuntimeWarning: Data file name in EEG.data (s6_sub-10007_rs_eeg.fdt) is incorrect, the file name must have changed on disk, using the correct file name (sub-40007_rs_eeg.fdt).
  data = mne.io.read_raw_eeglab(path, preload=True, verbose=False)


/data/zanola/BrainLats/3_PD/CL/sub-40007/eeg/sub-40007_rs_eeg.set 26_407_1_1 3_PD 407 (128, 202500)


...

/data/zanola/BrainLats/4_MS/AR/suj_533/eeg/suj_533_rs_eeg.set 26_533_1_1 4_MS 533 (128, 105250)


/tmp/ipykernel_2331099/1742952222.py:17: RuntimeWarning: Data file name in EEG.data (s6_sub-100023_rs_eeg.fdt) is incorrect, the file name must have changed on disk, using the correct file name (suj_534_rs_eeg.fdt).
  data = mne.io.read_raw_eeglab(path, preload=True, verbose=False)


/data/zanola/BrainLats/4_MS/AR/suj_534/eeg/suj_534_rs_eeg.set 26_534_1_1 4_MS 534 (128, 106750)


/tmp/ipykernel_2331099/1742952222.py:17: RuntimeWarning: Data file name in EEG.data (s6_sub-100024_rs_eeg.fdt) is incorrect, the file name must have changed on disk, using the correct file name (suj_535_rs_eeg.fdt).
  data = mne.io.read_raw_eeglab(path, preload=True, verbose=False)


/data/zanola/BrainLats/4_MS/AR/suj_535/eeg/suj_535_rs_eeg.set 26_535_1_1 4_MS 535 (128, 109250)
/data/zanola/BrainLats/5_HC/CL/sub-10001/eeg/s6_sub-10001_rs_eeg.set 26_101_1_1 5_HC 101 (128, 141954)
/data/zanola/BrainLats/5_HC/AR/sub-10002/eeg/s6_sub-10002_rs_eeg.set 26_102_1_1 5_HC 102 (128, 91106)
/data/zanola/BrainLats/5_HC/AR/sub-10003/eeg/s6_sub-10003_rs_eeg.set 26_103_1_1 5_HC 103 (128, 145163)
/data/zanola/BrainLats/5_HC/AR/sub-10004/eeg/s6_sub-10004_rs_eeg.set 26_104_1_1 5_HC 104 (128, 101771)
/data/zanola/BrainLats/5_HC/CL/sub-10005/eeg/s6_sub-10005_rs_eeg.set 26_105_1_1 5_HC 105 (128, 145339)
/data/zanola/BrainLats/5_HC/AR/sub-10006/eeg/s6_sub-10006_rs_eeg.set 26_106_1_1 5_HC 106 (128, 109321)
/data/zanola/BrainLats/5_HC/AR/sub-10007/eeg/s6_sub-10007_rs_eeg.set 26_107_1_1 5_HC 107 (128, 89363)
/data/zanola/BrainLats/5_HC/CL/sub-10008/eeg/s6_sub-10008_rs_eeg.set 26_108_1_1 5_HC 108 (128, 149574)
/data/zanola/BrainLats/5_HC/AR/sub-10009/eeg/s6_sub-10009_rs_eeg.set 26_109_1_1 5_

In [59]:
label

'1_AD'

In [ ]:
label = int(path.split(os.sep)[4][0])-1
label

In [ ]:
path.split(os.sep)[3]

In [70]:
        c_id = (list(range(1, 13)) +  # 001–012
               [14, 15, 16, 17, 18, 20, 21, 22, 24, 26, 28, 29, 30, 31, 33, 34, 35, 37, 38, 43])
        len(c_id)

32

In [71]:
b_id = list(range(101, 120))  # 101–119 inclusive
len(b_id)

19

In [73]:
(list(range(1, 13)) +  # 001–012
               [14, 15, 16, 17, 18, 20, 21, 22, 24, 26, 28, 29, 30, 31, 33, 34, 35, 37, 38, 43])

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 14,
 15,
 16,
 17,
 18,
 20,
 21,
 22,
 24,
 26,
 28,
 29,
 30,
 31,
 33,
 34,
 35,
 37,
 38,
 43]

In [75]:
c_id = [f"{i:03}" for i in (
    list(range(1, 13)) +
    [14, 15, 16, 17, 18, 20, 21, 22, 24, 26, 28, 29, 30, 31, 33, 34, 35, 37, 38, 43]
)]

In [76]:
c_id

['001',
 '002',
 '003',
 '004',
 '005',
 '006',
 '007',
 '008',
 '009',
 '010',
 '011',
 '012',
 '014',
 '015',
 '016',
 '017',
 '018',
 '020',
 '021',
 '022',
 '024',
 '026',
 '028',
 '029',
 '030',
 '031',
 '033',
 '034',
 '035',
 '037',
 '038',
 '043']